# astroVIPER Tutorial 2: Processing Set

In [ ]:
import os

from importlib.metadata import version

try:
    import astroviper

    print('astroVIPER version', version('astroviper'), 'already installed.')
except ImportError as e:
    print(e)
    print('Installing astroVIPER')

    os.system("pip install astroviper")

    import xradio
    print('astroVIPER version', version('astroviper'), ' installed.')

## Download and convert dataset

In [3]:
from xradio.data.datasets import download
download(file='Antennae_North.cal.lsrk.split.ms',source='dropbox')

./Antennae_North.cal.lsrk.split.ms.zip: 100%|█████████████████████████████████████████████████████████████████████████| 1.49M/1.49M [00:01<00:00, 1.49MiB/s]


## Setup Dask Cluster
To simplify things we are going to start of by just using a single thread (everything will run in serial).

In [3]:
import astroviper
from astroviper.client import local_client

viper_client = local_client(cores=2, memory_limit="4GB")
viper_client

# import dask
# dask.config.set(scheduler='synchronous')

{'log_to_term': True, 'log_to_file': False, 'log_file': 'viper_', 'log_level': 'INFO'}


2023-10-20 17:52:18 INFO     State start
2023-10-20 17:52:18 INFO       Scheduler at:     tcp://127.0.0.1:64020
2023-10-20 17:52:18 INFO       dashboard at:            127.0.0.1:8787
2023-10-20 17:52:18 INFO             Start Nanny at: 'tcp://127.0.0.1:64023'
2023-10-20 17:52:18 INFO             Start Nanny at: 'tcp://127.0.0.1:64024'
2023-10-20 17:52:19 INFO     Register worker <WorkerState 'tcp://127.0.0.1:64027', name: 0, status: init, memory: 0, processing: 0>
2023-10-20 17:52:19 INFO     Starting worker compute stream, tcp://127.0.0.1:64027
2023-10-20 17:52:19 INFO     Starting established connection to tcp://127.0.0.1:64030
2023-10-20 17:52:19 INFO     Register worker <WorkerState 'tcp://127.0.0.1:64028', name: 1, status: init, memory: 0, processing: 0>
2023-10-20 17:52:19 INFO     Starting worker compute stream, tcp://127.0.0.1:64028
2023-10-20 17:52:19 INFO     Starting established connection to tcp://127.0.0.1:64032
2023-10-20 17:52:19 INFO     Receive client connection: Clien

init local cache
log_parms {'log_to_term': False, 'log_to_file': False, 'log_file': 'viper_', 'log_level': 'INFO'}
[client.py:115 -         local_client() ] Created client <Client: 'tcp://127.0.0.1:64020' processes=2 threads=2, memory=7.45 GiB>


2023-10-20 17:52:19 INFO     Created client <Client: 'tcp://127.0.0.1:64020' processes=2 threads=2, memory=7.45 GiB>


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 2,Total memory: 7.45 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:64020,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 2
Started: Just now,Total memory: 7.45 GiB
Comm: tcp://127.0.0.1:64027,Total threads: 1
Dashboard: http://127.0.0.1:64029/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:64023,


## Inscpect Processing Set

The read_processing_set is a lazy function so no data is loaded into memory. 

In [4]:
ps_name = '/Users/jsteeb/Dropbox/Data/Antennae_North.cal.lsrk.split.vis.zarr'

from xradio.vis.read_processing_set import read_processing_set
#sel_parms = {'xds_id':[0,1,2]}
#fields = ["NGC4038 - Antennae North"]
intents = ["OBSERVE_TARGET#ON_SOURCE"]
fields = None
ps = read_processing_set(ps_name,
                        intents=intents,
                        fields=fields)
ps.summary()

,name,ddi,intent,field_id,field_name,start_frequency,end_frequency
0,Antennae_North.cal.lsrk.split_ddi_0_intent_OBS...,0,OBSERVE_TARGET#ON_SOURCE,1,NGC4038 - Antennae North,3.439281e+11,3.440067e+11
1,Antennae_North.cal.lsrk.split_ddi_0_intent_OBS...,0,OBSERVE_TARGET#ON_SOURCE,0,NGC4038 - Antennae North,3.439281e+11,3.440067e+11
2,Antennae_North.cal.lsrk.split_ddi_0_intent_OBS...,0,OBSERVE_TARGET#ON_SOURCE,2,NGC4038 - Antennae North,3.439281e+11,3.440067e+11


In [5]:
list(ps.items())

[('Antennae_North.cal.lsrk.split_ddi_0_intent_OBSERVE_TARGET#ON_SOURCE_field_id_1',
  <xarray.Dataset>
  Dimensions:                     (time: 35, baseline_id: 64, frequency: 8,
                                   polarization: 2, uvw_label: 3)
  Coordinates:
      baseline_antenna1_id        (baseline_id) int64 dask.array<chunksize=(64,), meta=np.ndarray>
      baseline_antenna2_id        (baseline_id) int64 dask.array<chunksize=(64,), meta=np.ndarray>
    * baseline_id                 (baseline_id) int64 0 1 2 3 4 ... 59 60 61 62 63
    * frequency                   (frequency) float64 3.439e+11 ... 3.44e+11
    * polarization                (polarization) <U2 'XX' 'YY'
    * time                        (time) float64 1.307e+09 1.307e+09 ... 1.307e+09
    * uvw_label                   (uvw_label) <U1 'u' 'v' 'w'
  Data variables:
      EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 dask.array<chunksize=(35, 64), meta=np.ndarray>
      FLAG                        (time, basel

## Inspect a single ms_v4

In [6]:
ps[list(ps.keys())[0]]  #Add method to PS get/quary for a specific msv4

<xarray.Dataset>
Dimensions:                     (time: 35, baseline_id: 64, frequency: 8,
                                 polarization: 2, uvw_label: 3)
Coordinates:
    baseline_antenna1_id        (baseline_id) int64 dask.array<chunksize=(64,), meta=np.ndarray>
    baseline_antenna2_id        (baseline_id) int64 dask.array<chunksize=(64,), meta=np.ndarray>
  * baseline_id                 (baseline_id) int64 0 1 2 3 4 ... 59 60 61 62 63
  * frequency                   (frequency) float64 3.439e+11 ... 3.44e+11
  * polarization                (polarization) <U2 'XX' 'YY'
  * time                        (time) float64 1.307e+09 1.307e+09 ... 1.307e+09
  * uvw_label                   (uvw_label) <U1 'u' 'v' 'w'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 dask.array<chunksize=(35, 64), meta=np.ndarray>
    FLAG                        (time, baseline_id, frequency, polarization) bool dask.array<chunksize=(35, 64, 3, 2), meta=np.ndarray>
    TIME_CENTROID               (time, baseline_id) float64 dask.array<chunksize=(35, 64), meta=np.ndarray>
    UVW                         (time, baseline_id, uvw_label) float64 dask.array<chunksize=(35, 64, 3), meta=np.ndarray>
    VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 dask.array<chunksize=(35, 64, 3, 2), meta=np.ndarray>
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 dask.array<chunksize=(35, 64, 3, 2), meta=np.ndarray>
Attributes:
    data_groups:  {'base': {'flag': 'FLAG', 'uvw': 'UVW', 'visibility': 'VISI...
    ddi:          0
    field_info:   {'code': 'none', 'delay_direction': {'attrs': {'frame': 'FK...
    intent:       OBSERVE_TARGET#ON_SOURCE
    antenna_xds:  <xarray.Dataset>\nDimensions:        (antenna_id: 15, coord...

## Creating Parallel Coordinates for the Map
The parallel coordinates determine the parallelism of the map graph. Each chunk in the parallel coordinates represents a selection criterion used for the subselected processing set that is sent to a node. There is one parallel coordinate chunk per node in the mapping part of the graph. 

The parallel_coords is a dictionary where each key is a dimension coordinate that appears in your data. For the ms_v4 the options would be time, baseline/antenna, frequency, polarization. The _make_parallel_coord function will convert any XRADIO measures into a parallel coordinate. In addition a convenious functions have been created _make_time_coord and _make_frequency_coord that will create numpy arrays.

In [7]:
ms_xds = ps[list(ps.keys())[1]]
ms_xds

<xarray.Dataset>
Dimensions:                     (time: 45, baseline_id: 64, frequency: 8,
                                 polarization: 2, uvw_label: 3)
Coordinates:
    baseline_antenna1_id        (baseline_id) int64 dask.array<chunksize=(64,), meta=np.ndarray>
    baseline_antenna2_id        (baseline_id) int64 dask.array<chunksize=(64,), meta=np.ndarray>
  * baseline_id                 (baseline_id) int64 0 1 2 3 4 ... 59 60 61 62 63
  * frequency                   (frequency) float64 3.439e+11 ... 3.44e+11
  * polarization                (polarization) <U2 'XX' 'YY'
  * time                        (time) float64 1.307e+09 1.307e+09 ... 1.307e+09
  * uvw_label                   (uvw_label) <U1 'u' 'v' 'w'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 dask.array<chunksize=(45, 64), meta=np.ndarray>
    FLAG                        (time, baseline_id, frequency, polarization) bool dask.array<chunksize=(45, 64, 3, 2), meta=np.ndarray>
    TIME_CENTROID               (time, baseline_id) float64 dask.array<chunksize=(45, 64), meta=np.ndarray>
    UVW                         (time, baseline_id, uvw_label) float64 dask.array<chunksize=(45, 64, 3), meta=np.ndarray>
    VISIBILITY                  (time, baseline_id, frequency, polarization) complex64 dask.array<chunksize=(45, 64, 3, 2), meta=np.ndarray>
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 dask.array<chunksize=(45, 64, 3, 2), meta=np.ndarray>
Attributes:
    data_groups:  {'base': {'flag': 'FLAG', 'uvw': 'UVW', 'visibility': 'VISI...
    ddi:          0
    field_info:   {'code': 'none', 'delay_direction': {'attrs': {'frame': 'FK...
    intent:       OBSERVE_TARGET#ON_SOURCE
    antenna_xds:  <xarray.Dataset>\nDimensions:        (antenna_id: 15, coord...

### Example: Frequency Parallel Coordinate

In [8]:
from astroviper._concurrency._graph_tools import _make_parallel_coord

parallel_coords = {}
n_chunks=3
parallel_coords['frequency'] = _make_parallel_coord(coord=ms_xds.frequency, n_chunks=n_chunks)
parallel_coords['frequency']

{'data': [343928096685.9587,
  343939328174.9401,
  343950559663.9216,
  343961791152.903,
  343973022641.88446,
  343984254130.8659,
  343995485619.84735,
  344006717108.8288],
 'data_chunks': {0: array([3.43928097e+11, 3.43939328e+11, 3.43950560e+11]),
  1: array([3.43961791e+11, 3.43973023e+11, 3.43984254e+11]),
  2: array([3.43995486e+11, 3.44006717e+11])},
 'dims': ('frequency',),
 'attrs': {'channel_width': {'attrs': {'type': 'quanta', 'units': ['Hz']},
   'data': 11231488.981445312,
   'dims': ''},
  'frame': 'LSRK',
  'reference_frequency': {'attrs': {'frame': 'LSRK',
    'type': 'spectral_coord',
    'units': ['Hz']},
   'data': 343928096685.9587,
   'dims': ''},
  'spectral_window_name': '',
  'spw_id': 0,
  'type': 'spectral_coord',
  'units': ['Hz']}}

2023-10-20 17:56:03 INFO     Closing Nanny gracefully at 'tcp://127.0.0.1:64023'. Reason: worker-close
2023-10-20 17:56:03 INFO     Closing Nanny gracefully at 'tcp://127.0.0.1:64024'. Reason: worker-close
2023-10-20 17:56:03 INFO     Worker process 15842 was killed by signal 15
2023-10-20 17:56:03 INFO     Worker process 15841 was killed by signal 15
2023-10-20 17:56:03 INFO     Closing Nanny at 'tcp://127.0.0.1:64024'. Reason: nanny-close-gracefully
2023-10-20 17:56:03 INFO     Closing Nanny at 'tcp://127.0.0.1:64023'. Reason: nanny-close-gracefully
2023-10-20 17:56:03 INFO     Connection to tcp://127.0.0.1:64032 has been closed.
2023-10-20 17:56:03 INFO     Remove worker <WorkerState 'tcp://127.0.0.1:64028', name: 1, status: closing, memory: 0, processing: 0>
2023-10-20 17:56:03 INFO     Removing comms to tcp://127.0.0.1:64028
2023-10-20 17:56:03 INFO     Connection to tcp://127.0.0.1:64030 has been closed.
2023-10-20 17:56:03 INFO     Remove worker <WorkerState 'tcp://127.0.0.1:640

## Create a chunk function and map graph

In [ ]:
from astroviper._concurrency._graph_tools import _map
import dask

def my_func(input_parms):
    print('Keys in input parameters: ',list(input_parms.keys()))
    print('Test_input:', input_parms['test_input'])
    print('data_sel',input_parms['data_sel'])
    print('chunk_coords',input_parms['chunk_coords'])
    print('chunk_indices',input_parms['chunk_indices'])
    print('chunk_id',input_parms['chunk_id'])
    print('*****'*10)
    return input_parms['test_input']

#['test_input', 'input_data_name', 'viper_local_dir', 'date_time', 'data_sel', 'chunk_coords', 'chunk_indx', 'chunk_id', 'parallel_dims']
input_parms= {}
input_parms['test_input'] = 42

ps_name = '/Users/jsteeb/Dropbox/Data/Antennae_North.cal.lsrk.split.vis.zarr'
sel_parms = {}
sel_parms["fields"] = ["NGC4038 - Antennae North"]
sel_parms["intents"] = ["OBSERVE_TARGET#ON_SOURCE"]
graph = _map(
    input_data_name = ps_name,
    input_data_type = 'processing_set',
    ps_sel_parms=sel_parms,
    parallel_coords=parallel_coords,
    func_chunk=my_func,
    input_parms=input_parms,
    client=None,
)

dask.visualize(graph)

## Run the graph

In [ ]:
dask.compute(graph)

## Adding a reduce

In [ ]:
from astroviper._concurrency._graph_tools import _reduce

def my_sum(input_1, input_2, input_parms):
    return input_1 + input_2 + input_parms['test_input']

input_parms= {}
input_parms['test_input'] = 5
graph_reduce = _reduce(graph, my_sum, input_parms)
dask.visualize(graph_reduce)

## Run the graph

In [ ]:
dask.compute(graph_reduce) 

### Example: Time Parallel Coordinate

In [ ]:
from astroviper._concurrency._graph_tools import  _make_time_coord, _make_parallel_coord

parallel_coord = {}
time_coord= _make_time_coord(time_start="2011-05-28T01:47:10.176",time_delta=6.048,n_samples=45,time_scale="utc")
time_coord

In [ ]:
_make_parallel_coord(coord=time_coord, n_chunks=3)

Using _make_time_coord is in general not very useful since it makes a series with a uniform interval (time_delta). We can also just use the time coordinates of the processing set.

In [ ]:
time_parallel_coord = {} 
time_parallel_coord['data'] = xr.concat([value.time.values for value in ps.values()],dim='time')


# time_coord = ps[list(ps.keys())[1]]
# import xarray as xr
# value_list = xr.concat([value.time for value in ps.values()],dim='time')
# #_make_parallel_coord(coord=time_coord, n_chunks=3)
# value_list

In [ ]:
n_chunks=3

coord = _make_frequency_coord(freq_start=343018346078.4616,
#     freq_delta=11231488.981445312*(166/80),
#     n_channels=80,)

parallel_coords['frequency'] = _make_parallel_coord(coord=ms_xds.frequency, n_chunks=n_chunks)
print(parallel_coords['frequency'])

- Baseline/antenna selections can only be done if the all the ms_v4's antenna ids have been synced, this is the case if the processing set was created from a single ms (in the future we will have a function that can merge processing sets and ensure that the ids are synced).
- Parallel coordinate chunks can overlap.
- Parallel coordinate values do have not have to be regular spaced.